# Standard Technical Indicators: Discrete Signal Entry/Exit


**Strategy Logic v6 (Signal Driven) with Intraday Stop Loss**:
- **Engine Mode**: `strict_signals=True`.
- **Signal Protocol**: 
    - `> 0`: Buy Entry.
    - `< 0`: Sell Exit.
    - `0.0`: Hold.
- **Stop Loss**: Intraday Simulation using OHLC data.
    - If `Low_t < Stop_Price`, trigger sell.
    - Sell Price = `Open_t` (if Gap Down) or `Stop_Price` (if Intraday).


In [ ]:
import os, sys

# --- Setup correct working directory (ROOT) ---
if os.getcwd().endswith('notebooks'):
    os.chdir('..')

ROOT = os.getcwd()
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

import numpy as np
import pandas as pd
from IPython.display import display
from bokeh.io import output_notebook, show

from src.backtester.data import load_cleaned_assets, align_close_prices
from src.backtester.engine import BacktestConfig, run_backtest
from src.backtester.metrics import compute_performance_stats
from src.backtester.bokeh_plots import build_interactive_portfolio_layout
from src.backtester.report import compute_backtest_report
from src.features.core import (
    adx, aroon, atr, bollinger_bands, cci, ema, fib_levels, ichimoku, macd, 
    obv, accumulation_distribution_line, roc, rsi, sma, stochastic_oscillator
)

output_notebook()


In [ ]:
cfg = BacktestConfig(
    initial_equity=1_000_000,
    transaction_cost_bps=5,
    rebalance=None,
    mode='event_driven',
    strict_signals=True,
    stop_loss_pct=0.15   # 15% Stop Loss (Reduced exit noise for volatile assets)
)


In [ ]:
assets = load_cleaned_assets(symbols=None)
close = align_close_prices(assets)

# Build OHLC Arrays for Backtester
def _align_col(assets, col):
    parts = []
    for sym, df in assets.items():
        parts.append(df[col].astype(float).rename(sym))
    return pd.concat(parts, axis=1).sort_index()

open_prices = _align_col(assets, 'Open')
high_prices = _align_col(assets, 'High')
low_prices = _align_col(assets, 'Low')

print(f'Loaded {len(assets)} assets with full OHLC.')
market_df = None

# Calculate Equal-Weight Benchmark (Long Only Buy & Hold)
ew_weights = pd.DataFrame(1.0 / len(assets), index=close.index, columns=close.columns)
benchmark_res = run_backtest(close, ew_weights, BacktestConfig(initial_equity=cfg.initial_equity, transaction_cost_bps=0))
benchmark_stats = compute_performance_stats(equity=benchmark_res.equity, returns=benchmark_res.returns)
print(f'Benchmark Return: {benchmark_stats.total_return*100:.2f}%')


In [ ]:
ALLOC = 0.01 

def smooth_slope(series: pd.Series, window: int = 20) -> pd.Series:
    return series.diff(1).ewm(span=window).mean()

def hysteresis_position(enter: pd.Series, exit: pd.Series) -> pd.Series:
    """Latches a signal state: +1.0 for Long, -1.0 for Out/Exit."""
    pos = pd.Series(0.0, index=enter.index)
    current = -1.0
    for dt in enter.index:
        if bool(exit.loc[dt]):
            current = -1.0
        elif bool(enter.loc[dt]):
            current = ALLOC
        pos.loc[dt] = current
    return pos

def get_divergence_signals(price: pd.Series, indicator: pd.Series, window: int = 20) -> pd.Series:
    p_slope = smooth_slope(price, window)
    i_slope = smooth_slope(indicator, window)
    signals = pd.Series(0.0, index=price.index)
    bull_div = (p_slope < 0) & (i_slope > 0)
    bear_div = (p_slope > 0) & (i_slope < 0)
    return hysteresis_position(bull_div, bear_div)

def build_market_proxy_ohlcv(assets: dict[str, pd.DataFrame], index: pd.DatetimeIndex) -> pd.DataFrame:
    def _col(name: str) -> pd.DataFrame:
        parts = []
        for sym, df in assets.items():
            s = df[name].astype(float).reindex(index)
            parts.append(s.rename(sym))
        return pd.concat(parts, axis=1)
    opens = _col('Open').mean(axis=1)
    highs = _col('High').mean(axis=1)
    lows = _col('Low').mean(axis=1)
    closes = _col('Close').mean(axis=1)
    vols = _col('Volume').sum(axis=1)
    return pd.DataFrame({'Open': opens, 'High': highs, 'Low': lows, 'Close': closes, 'Volume': vols})
market_df = build_market_proxy_ohlcv(assets, close.index)


## 1. On-Balance Volume


In [ ]:
def strategy_obv_discrete(assets): 
    out = {}
    for sym, df in assets.items():
        out[sym] = get_divergence_signals(df['Close'], obv(df['Close'], df['Volume']))
    return pd.DataFrame(out)


## 2. A/D Line


In [ ]:
def strategy_ad_discrete(assets):
    out = {}
    for sym, df in assets.items():
        ad_v = accumulation_distribution_line(df['High'], df['Low'], df['Close'], df['Volume'])
        out[sym] = get_divergence_signals(df['Close'], ad_v)
    return pd.DataFrame(out)


## 3. RSI


In [ ]:
def strategy_rsi_discrete(assets):
    out = {}
    for sym, df in assets.items():
        r = rsi(df['Close'], 14)
        enter = (r < 30.0)
        exit = (r > 70.0)
        out[sym] = hysteresis_position(enter, exit)


## 4. MACD


In [ ]:
def strategy_macd_discrete(assets):
    out = {}
    for sym, df in assets.items():
        m = macd(df['Close'])
        diff = m['macd'] - m['macd_signal']
        # Use hysteresis to prevent daily whipsaws on line crosses
        enter = (diff > 0) 
        exit = (diff < 0)
        out[sym] = hysteresis_position(enter, exit)
    return pd.DataFrame(out)


## 5. Stochastic


In [ ]:
def strategy_stochastic_discrete(assets):
    out = {}
    for sym, df in assets.items():
        st = stochastic_oscillator(df['High'], df['Low'], df['Close'])
        k = st['stoch_k']
        enter = (k < 20.0)
        exit = (k > 80.0)
        out[sym] = hysteresis_position(enter, exit)
    return pd.DataFrame(out)


## 6. ADX


In [ ]:
def strategy_adx_discrete(assets):
    out = {}
    for sym, df in assets.items():
        a = adx(df['High'], df['Low'], df['Close'])
        enter = (a['adx'] > 25) & (a['plus_di'] > a['minus_di'])
        exit = (a['minus_di'] > a['plus_di']) | (a['adx'] < 20)
        out[sym] = hysteresis_position(enter, exit)
    return pd.DataFrame(out)


## 7. Aroon


In [ ]:
def strategy_aroon_discrete(assets):
    out = {}
    for sym, df in assets.items():
        ar = aroon(df['High'], df['Low'])
        enter = (ar['aroon_up'] > 70)
        exit = (ar['aroon_up'] < 30)
        out[sym] = hysteresis_position(enter, exit)
    return pd.DataFrame(out)


## 8. CCI


In [ ]:
def strategy_cci_discrete(assets):
    out = {}
    for sym, df in assets.items():
        c_val = cci(df['High'], df['Low'], df['Close'])
        enter = (c_val > 100.0)
        exit = (c_val < -100.0)
        out[sym] = hysteresis_position(enter, exit)
    return pd.DataFrame(out)


## 9. Bollinger Bands


In [ ]:
def strategy_bb_discrete(assets):
    out = {}
    for sym, df in assets.items():
        bb = bollinger_bands(df['Close'])
        c = df['Close']
        enter = (c > bb['bb_upper'])
        exit = (c < bb['bb_mid'])
        out[sym] = hysteresis_position(enter, exit)
    return pd.DataFrame(out)


## 10. Ichimoku Cloud


In [ ]:
def strategy_ichimoku_discrete(assets):
    out = {}
    for sym, df in assets.items():
        ich = ichimoku(df['High'], df['Low'], df['Close'])
        c = df['Close']
        span_a = ich['ichimoku_span_a']
        span_b = ich['ichimoku_span_b']
        cloud_top = pd.concat([span_a, span_b], axis=1).max(axis=1)
        cloud_bottom = pd.concat([span_a, span_b], axis=1).min(axis=1)
        enter = (c > cloud_top)
        exit = (c < cloud_bottom)
        out[sym] = hysteresis_position(enter, exit)
    return pd.DataFrame(out)


## Execution


### OBV Native Discrete


In [ ]:
print('Running OBV Native Discrete...')
signals = strategy_obv_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='OBV Native Discrete'
)
show(layout)


### A/D Native Discrete


In [ ]:
print('Running A/D Native Discrete...')
signals = strategy_ad_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='A/D Native Discrete'
)
show(layout)


### RSI Native Discrete


In [ ]:
print('Running RSI Native Discrete...')
signals = strategy_rsi_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='RSI Native Discrete'
)
show(layout)


### MACD Native Discrete


In [ ]:
print('Running MACD Native Discrete...')
signals = strategy_macd_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='MACD Native Discrete'
)
show(layout)


### Stochastic Native Discrete


In [ ]:
print('Running Stochastic Native Discrete...')
signals = strategy_stochastic_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='Stochastic Native Discrete'
)
show(layout)


### ADX Native Discrete


In [ ]:
print('Running ADX Native Discrete...')
signals = strategy_adx_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='ADX Native Discrete'
)
show(layout)


### Aroon Native Discrete


In [ ]:
print('Running Aroon Native Discrete...')
signals = strategy_aroon_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='Aroon Native Discrete'
)
show(layout)


### CCI Native Discrete


In [ ]:
print('Running CCI Native Discrete...')
signals = strategy_cci_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='CCI Native Discrete'
)
show(layout)


### Bollinger Bands Native Discrete


In [ ]:
print('Running Bollinger Bands Native Discrete...')
signals = strategy_bb_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='Bollinger Bands Native Discrete'
)
show(layout)


### Ichimoku Native Discrete


In [ ]:
print('Running Ichimoku Native Discrete...')
signals = strategy_ichimoku_discrete(assets).reindex(close.index)
# Pass OHLC Data to Engine for Stop Loss Simulation
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='Ichimoku Native Discrete'
)
show(layout)
